### Get File Size

In [1]:
from pathlib import Path

file_path = "boericke_full_text.txt"

path = Path(file_path)
size_bytes = path.stat().st_size

size_kb = size_bytes / 1024
size_mb = size_bytes / (1024 ** 2)

print(f"Size in MB: {size_mb:.3f}")


Size in MB: 1.241


In [2]:
from time import time

In [3]:
from rich import print
from rich.console import Console

console = Console(width=140)

### Read and Load File Contents

In [4]:
with open(file_path, "r", encoding="utf-8") as f:
    text = f.read()

print(text[:100])

ABIES CANADENSIS
Pinus canadensis
Hemlock Spruce
Mucous membranes are affected by Abies can and gast

# Document Structure Aware

## 1. Sentence-based

### NLTK

In [5]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/srijanshovit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
def sentence_based_chunking(text):
    sentences = sent_tokenize(text)
    return [sentence.strip() for sentence in sentences]

In [7]:
start = time()
chunks = sentence_based_chunking(text)
end = time()
print(f"Time taken to chunk with custom code: {(end - start):.3f} s")
print(f"Total no of chunks: {len(chunks)}")

Time taken to chunk with custom code: 0.191 s

Total no of chunks: 26492

In [8]:
for i, chunk in enumerate(chunks[:5]):
    print(f"Chunk {i+1}: {chunk}")

Chunk 1: ABIES CANADENSIS
Pinus canadensis
Hemlock Spruce
Mucous membranes are affected by Abies can and gastric symptoms are most marked, and a catarrhal condition of the 
stomach is produced.

Chunk 2: There are peculiar cravings and chilly sensations that are very characteristic, especially for women with 
uterine displacement, probably due to defective nutrition with debility.

Chunk 3: Respiration and heart action labored.

Chunk 4: Wants to lie down all the time; skin cold and clammy, hands cold; very faint.

Chunk 5: Right lung and liver feel small and hard.

### SpaCy

In [9]:
import spacy

nlp = spacy.blank("en")
nlp.max_length = len(text) + 1000
nlp.add_pipe("sentencizer")

start = time()
doc = nlp(text)
sentences = [s.text for s in doc.sents]
end = time()
print(f"Time taken to chunk with custom code: {(end - start):.3f} s")
print(f"Total no of chunks: {len(chunks)}")



Time taken to chunk with custom code: 1.266 s

Total no of chunks: 26492

In [10]:
for i, chunk in enumerate(chunks[:5]):
    print(f"Chunk {i+1}: {chunk}")

Chunk 1: ABIES CANADENSIS
Pinus canadensis
Hemlock Spruce
Mucous membranes are affected by Abies can and gastric symptoms are most marked, and a catarrhal condition of the 
stomach is produced.

Chunk 2: There are peculiar cravings and chilly sensations that are very characteristic, especially for women with 
uterine displacement, probably due to defective nutrition with debility.

Chunk 3: Respiration and heart action labored.

Chunk 4: Wants to lie down all the time; skin cold and clammy, hands cold; very faint.

Chunk 5: Right lung and liver feel small and hard.

### Llama-Index

##### Sentence Splitter

#### Boundary tokens overlap

In [11]:
from llama_index.core.utils import get_tokenizer

tokenizer = get_tokenizer()

In [12]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(
    chunk_size=256,
    chunk_overlap=50,
    separator="."
)

start = time()
chunks = splitter.split_text(text)
end = time()
print(f"Time taken to chunk with custom code: {(end - start):.3f} s")

Time taken to chunk with custom code: 0.562 s

In [13]:
print(f"Total no of chunks: {len(chunks)}")
for i, c in enumerate(chunks[:3]):
    console.print(f"Chunk {i+1}:\n{c}")
    console.print(f"No of chars in the chunk: {len(c)}")

    n_tokens = len(tokenizer(c))
    console.print(f"No of tokens in the chunk: {n_tokens}")

Total no of chunks: 1716

Chunk 1:
ABIES CANADENSIS
Pinus canadensis
Hemlock Spruce
Mucous membranes are affected by Abies can and gastric symptoms are most marked, and a catarrhal condition of the stomach is produced. There
are peculiar cravings and chilly sensations that are very characteristic, especially for women with uterine displacement, probably due to 
defective nutrition with debility.
Respiration and heart action labored. Wants to lie down all the time; skin cold and clammy, hands cold; very faint. Right lung and liver 
feel small and hard.
Gleet.
Head.––Feels light-headed, tipsy. Irritable.
Stomach.––Canine hunger with torpid liver. Gnawing, hungry, faint feeling at the epigastrium. Great appetite, craving for meat, pickles, 
radishes, turnips, artichokes, coarse food. Tendency to eat far beyond capacity for digestion.
Burning and distention of stomach and abdomen with palpitation. Flatulence disturbs the heart's action. Pain in right shoulder-blade, and 
constipation, with burning in rectum.
Female.––Uterine displacements.

No of chars in the chunk: 1012

No of tokens in the chunk: 240

Chunk 2:
Flatulence disturbs the heart's action. Pain in right shoulder-blade, and constipation, with burning in rectum.
Female.––Uterine displacements. Sore feeling at fundus of uterus, relieved by pressure. Prostration; wants to lie down all the time. Thinks 
womb is soft and feeble.
Fever.––Cold shivering, as if blood were ice-water (Acon). Chills run down back.
Cold water feeling between shoulders (Ammon mur). Skin clammy and sticky.
Night-sweat (China).
Dose.––First to third potency.
ABIES NIGRA
Black Spruce
A powerful and long-acting remedy, in various forms of disease, whenever the characteristic stomach symptoms are present. Most of the 
symptoms are associated with the gastric disturbances. In dyspeptic troubles of the aged, with functional heart symptoms; also after tea or 
tobacco. Constipation. Pain in external meatus.
Head.––Hot, with flushed cheeks. Low-spirited. Dull during the day, wakeful at night. Unable to think.
Stomach.––Pain in stomach always comes on after eating.

No of chars in the chunk: 989

No of tokens in the chunk: 240

Chunk 3:
Head.––Hot, with flushed cheeks. Low-spirited. Dull during the day, wakeful at night. Unable to think.
Stomach.––Pain in stomach always comes on after eating. Sensation of a lump that hurts, as if a hard-boiled egg had lodged in cardiac end of
stomach; continual distressing constriction just above the pit of the stomach, as if everything were knotted up. Total loss of appetite in 
morning, but great craving for food at noon and night. Offensive breath. Eructations.
Chest.––Painful sensation, as if something were lodged in the chest and had to be coughed up; lungs feel compressed. Cannot be fully 
expanded. Worse coughing; waterbrash succeeds cough. Choking sensation in throat. Dyspnœa; worse lying down; sharp, cutting pain in heart; 
heart's action heavy and slow; tachycardia, bradycardia.
Back.––Pain in small of back. Rheumatic pains and aching in bones.
Sleep.––Wakeful and restless at night, with hunger. Bad dreams.

No of chars in the chunk: 928

No of tokens in the chunk: 232

### SentenceWindowNodeParser

#### Boundary Sentences Overlap

In [14]:
from llama_index.core import Document
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.utils import get_tokenizer

doc = Document(text=text)


In [15]:
window_size = 1

In [16]:
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=window_size,
    window_metadata_key="window",
    original_text_metadata_key="original_sentence",
)


In [17]:
start = time()
nodes = node_parser.get_nodes_from_documents([doc])
end = time()
print(f"Time taken to chunk with custom code: {(end - start):.3f} s")
print(f"Total no of nodes (sentences): {len(nodes)}")


Time taken to chunk with custom code: 0.871 s

Total no of nodes (sentences): 26550

In [18]:
tokenizer = get_tokenizer()
tokenizer

functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all')

### What is cl100k_base?

cl100k_base is a Byte Pair Encoding (BPE) tokenizer designed for ~100k vocabulary scale and used by modern OpenAI-family models.

- cl → Chat / Completion lineage
- 100k → ~100,000 merge vocabulary
- base → base encoding (no chat-specific wrapping like <|system|>,<|user|>,<|assistant|>)

In [19]:


for i, node in enumerate(nodes[:3], start=len(nodes) - 4):
    center = node.text
    window = node.metadata["window"]

    print(f"Node {i}")
    print("CENTER SENTENCE:")
    print(center)

    print(f"WINDOW (±{window_size} sentences):")
    print(window)

    print("Stats:")
    print(f"tokens (center): {len(tokenizer(center))}; tokens (window): {len(tokenizer(window))}")



Node 26546

CENTER SENTENCE:

ABIES CANADENSIS
Pinus canadensis
Hemlock Spruce
Mucous membranes are affected by Abies can and gastric symptoms are most marked, and a catarrhal condition of the 
stomach is produced.

WINDOW (±1 sentences):

ABIES CANADENSIS
Pinus canadensis
Hemlock Spruce
Mucous membranes are affected by Abies can and gastric symptoms are most marked, and a catarrhal condition of the 
stomach is produced.  There are peculiar cravings and chilly sensations that are very characteristic, especially 
for women with uterine displacement, probably due to defective nutrition with debility.

...

Respiration and heart action labored.

WINDOW (±1 sentences):

There are peculiar cravings and chilly sensations that are very characteristic, especially for women with uterine 
displacement, probably due to defective nutrition with debility.
 Respiration and heart action labored.  Wants to lie down all the time; skin cold and clammy, hands cold; very 
faint.

Stats:

tokens (center): 9; tokens (window): 60